In [1]:
import pickle
from timeit import default_timer as timer

import os
import platform
import tensorflow as tf
import keras;

# *print ("Current file path: {}".format(os.path.dirname(os.path.realpath(__file__))))
print ("Current Working Directory: {}".format(os.getcwd()))
print("Python: {}, TensorFlow:{}, Keras:{}".\
      format(platform.python_version(), tf.__version__, keras.__version__))

random_seed = 1



Using TensorFlow backend.


Current Working Directory: /home/endre/git/finance_ml/src
Python: 3.6.1, TensorFlow:1.2.1, Keras:2.0.5


# Load pickled Features, Labels and Name of each Feature-array

In [2]:
print ("De-pickling features, labels and feature/label names..")
load_start = timer()

# pickled = pickle.load(open("RangeNamesFeaturesAndLabels-mk1.pickle", "rb"))
# pickled = pickle.load(open("RangeNamesFeaturesAndLabels-smallSet-6stocks.pickle", "rb"))
pickled = pickle.load(open("RangeNamesFeaturesAndLabels-smallSet-50stocks.pickle", "rb"))
# pickled = pickle.load(open("RangeNamesFeaturesAndLabels-mediumset-unknownNumStocks.pickle", "rb"))
# pickled = pickle.load(open("RangeNamesFeaturesAndLabels-largeSet-1619stocks.pickle", "rb"))

load_millis = (timer()-load_start) * 1000
print ("De-pickle took {} ms".format(load_millis))

# {'rangeNames': rangeNames, 'features': features, 'labels': labels}
rangeNames = pickled['rangeNames']
features = pickled['features']
labels = pickled['labels']
print("\nRangeNames/features/labels len: {}/{}/{}".\
      format(len(rangeNames), len(features), len(labels)))
numFeatures = len(features[0])
print("\nNumber of Features for each feature-array: {}".format(numFeatures))
print("Number of Labels for each label-array: {}".format(len(labels[0])))

De-pickling features, labels and feature/label names..
De-pickle took 1806.1965110246092 ms

RangeNames/features/labels len: 71036/71036/71036

Number of Features for each feature-array: 54
Number of Labels for each label-array: 4


# Split set into Train and Test

In [3]:
import resource
import gc
def mem():
    print('Memory usage: %2.2f MB' % round(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0,1))
    
mem()
trainRangeNames = []
trainFeatures = []
trainLabels = []

testRangeNames = []
testFeatures = []
testLabels = []
mem()

gc.collect()
mem()

length = len(rangeNames)
for i in range(length):
    names = rangeNames[i]
    if (names[1] < '2016-01-01'):
        trainRangeNames.append(names)
        trainFeatures.append(features[i])
        trainLabels.append(labels[i])
    else:
        testRangeNames.append(names)
        testFeatures.append(features[i])
        testLabels.append(labels[i])

lenTrainFeatures = len(trainFeatures)

print("TRAIN RangeNames/features/labels len: {}/{}/{} - {:.2f}% of total".\
      format(len(trainRangeNames), lenTrainFeatures, len(trainLabels), (lenTrainFeatures / len(features)) * 100 ))
print("TEST RangeNames/features/labels len: {}/{}/{} - {:.2f}% of total".\
      format(len(testRangeNames), len(testFeatures), len(testLabels), (len(testFeatures) / len(features)) * 100 ))

print("\nTotal loaded features: {}, trainFeatures + testFeatures:{} - {:.4f}%".\
      format(len(features), lenTrainFeatures + len(testFeatures), ((lenTrainFeatures + len(testFeatures)) / len(features)) * 100))

Memory usage: 811.70 MB
Memory usage: 811.70 MB
Memory usage: 811.70 MB
TRAIN RangeNames/features/labels len: 66310/66310/66310 - 93.35% of total
TEST RangeNames/features/labels len: 4726/4726/4726 - 6.65% of total

Total loaded features: 71036, trainFeatures + testFeatures:71036 - 100.0000%


In [4]:
# One-hot the labels

# 0:  5 days
# 1: 10 days
# 2: 15 days
# 3: 20 days
label_to_use = 3

good = [1,0]
bad =  [0,1]

trainLabels_onehot = [good if x[label_to_use] > 0 else bad  for x in trainLabels]
testLabels_onehot = [good if x[label_to_use] > 0 else bad  for x in testLabels]
print("One-hotted ok.")

sum_trainLabels = [sum(i) for i in zip(*trainLabels_onehot)]
sum_testLabels = [sum(i) for i in zip(*testLabels_onehot)]
print("Train labels [good, bad]: {} -> [{:.2f}%, {:.2f}%]".\
      format(sum_trainLabels, (sum_trainLabels[0] / len(trainLabels)) * 100, (sum_trainLabels[1] / len(trainLabels)) * 100))
print("Test labels  [good, bad]: {} -> [{:.2f}%, {:.2f}%]".\
      format(sum_testLabels, (sum_testLabels[0] / len(testLabels)) * 100, (sum_testLabels[1] / len(testLabels)) * 100))

One-hotted ok.
Train labels [good, bad]: [35957, 30353] -> [54.23%, 45.77%]
Test labels  [good, bad]: [2849, 1877] -> [60.28%, 39.72%]


# !! RUN FROM HERE WHEN ONLY TWEAKING NETWORK !!

In [46]:
from sklearn.utils import shuffle
import numpy as np

currentTrainPos = 0
current_shuffle = 0
shuff_trainFeatures = []
shuff_trainLabels_onehot = []
def getTrainMiniBatch(miniBatchSize):
    global currentTrainPos, current_shuffle
    global shuff_trainFeatures, shuff_trainLabels_onehot
    # If we'll overflow the features list with this mini_batch, then reset the start position
    if (currentTrainPos + miniBatchSize > lenTrainFeatures):
        currentTrainPos = 0

    # If the start position is 0, then run shuffling of features and labels.
    if (currentTrainPos == 0):
        print("Shuffling training features and labels - {}..".format(current_shuffle))
        shuff_trainFeatures, shuff_trainLabels_onehot = shuffle(trainFeatures, trainLabels_onehot, random_state=current_shuffle)
        current_shuffle += 1

    # Pick out the desired mini_batch size.
    start = currentTrainPos
    end = currentTrainPos + miniBatchSize
    # .. store this mini_batch end pos as the next mini_batch start pos.
    currentTrainPos = end
    
    # return np.array(shuff_trainFeatures[start:end]) / 100.0, np.array(shuff_trainLabels_onehot[start:end])
    return shuff_trainFeatures[start:end], shuff_trainLabels_onehot[start:end]

# Set up Neural Network

In [47]:
n1=800
n2=1600
n3=800
# n4=600
out=2
epochs = 100 # Not explicitly used, as we count mini_batches
mini_batch_size = 128
learning_rate = 0.001  # 0.0001 is good, but maybe use higher learning rate when dropout'ing..
keep_prob_in_train = 1
keep_prob_hid_train = 1
transfer_function = tf.nn.relu
l2_regularization_beta = 0 # This seemed good, but currently having all softmaxed_test_Y to [ 0.53961992,  0.46038002]: 0.025

#========
mini_batches_per_epoch = int(lenTrainFeatures / mini_batch_size)
total_mini_batches = mini_batches_per_epoch * epochs

print("Mini batches per epoch: trainFeats {} / mini_batch_size {} = {}".format(lenTrainFeatures, mini_batch_size, mini_batches_per_epoch))
print("Total mini batches: mini_batches_per_epoch {} * epochs {} = {}".format(mini_batches_per_epoch, epochs, total_mini_batches))

# Reset the default graph, so as to chuck out existing variables
tf.reset_default_graph()

with tf.variable_scope("input"):
    X = tf.placeholder(tf.float32, shape=[None, numFeatures], name="X")
    keep_prob_in = tf.placeholder(tf.float32, name="keep_prob")
    X_drop = tf.nn.dropout(X, keep_prob_in)
    labels = tf.placeholder(tf.float32, shape=[None, out], name="labels")

with tf.variable_scope("hidden_layers_common"):
    keep_prob_hid = tf.placeholder(tf.float32, name="keep_prob")

def weight_variable(name, shape):
    # return tf.get_variable(name, initializer=tf.glorot_uniform_initializer(seed=random_seed), shape=shape)
    return tf.get_variable(name, initializer=tf.contrib.layers.xavier_initializer(), shape=shape)
    
def bias_variable(name, shape):
    # return tf.get_variable(name, initializer=tf.glorot_uniform_initializer(seed=random_seed), shape=shape)
    # return tf.get_variable(name, initializer=tf.contrib.layers.xavier_initializer(), shape=shape)
    return tf.get_variable(name, initializer=tf.constant_initializer(0.1), shape=shape)

with tf.variable_scope("layer1"):
    w1 = weight_variable("weight", [numFeatures, n1])
    b1 = bias_variable("bias", [n1])
    l1 = tf.matmul(X_drop, w1) + b1
    l1 = transfer_function(l1)
    l1 = tf.nn.dropout(l1, keep_prob_hid)

with tf.variable_scope("layer2"):
    w2 = weight_variable("weight", [n1, n2])
    b2 = bias_variable("bias", [n2])
    l2 = tf.matmul(l1, w2) + b2
    l2 = transfer_function(l2)
    l2 = tf.nn.dropout(l2, keep_prob_hid)

with tf.variable_scope("layer3"):
    w3 = weight_variable("weight", [n2, n3])
    b3 = bias_variable("bias", [n3])
    l3 = tf.matmul(l2, w3) + b3
    l3 = transfer_function(l3)
    l3 = tf.nn.dropout(l3, keep_prob_hid)

#with tf.variable_scope("layer4"):
#    w4 = weight_variable("weight", [n3, n4])
#    b4 = bias_variable("bias", [n4])
#    l4 = tf.matmul(l3, w4) + b4
#    l4 = transfer_function(l4)
    # l4 = tf.nn.dropout(l4, keep_prob_hid)

with tf.variable_scope("output"):
    wy = weight_variable("weight", [n3, out])
    by = bias_variable("bias", [out])
    Y = tf.matmul(l3, wy) + by

# :: Loss function with L2 Regularization
#l2_regularizer = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w3) + \
#                 tf.nn.l2_loss(w4) + tf.nn.l2_loss(wy)
# NOTE: L1 regularizer: tf.reduce_sum(tf.abs(tensor)) instead of tf.nn.l2_loss(tensor)
# Read here: https://github.com/tensorflow/models/blob/master/inception/inception/slim/losses.py
# Max-norm:
# https://stackoverflow.com/questions/37801832/how-can-i-implement-max-norm-constraints-in-an-mlp-in-tensorflow
# Hmm..: https://stackoverflow.com/questions/34934303/renormalize-weight-matrix-using-tensorflow

# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y, labels=labels) +\
#                      l2_regularization_beta * l2_regularizer)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y, labels=labels))

# :: Optimizer/Trainer
optimizer = tf.train.AdamOptimizer(learning_rate)
# optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)

# Define Test/Evaluate: Accuracy: Fraction right predictions
argmaxed_Y = tf.argmax(Y, 1)
correct_prediction = tf.equal(argmaxed_Y, tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Make Softmax node, to use for evaluation
softmaxed_Y = tf.nn.softmax(Y)

print("Network set up.")

Mini batches per epoch: trainFeats 66310 / mini_batch_size 128 = 518
Total mini batches: mini_batches_per_epoch 518 * epochs 100 = 51800
Network set up.


# Train Neural Network

In [48]:
check_length = len(testFeatures)
check_trainFeatures, check_trainLabels_onehot = shuffle(trainFeatures, trainLabels_onehot, random_state=42)
check_trainFeatures = check_trainFeatures[:check_length]
check_trainLabels_onehot = check_trainLabels_onehot[:check_length]

sess = tf.Session()

# Initialize TensorFlow variables in Session
sess.run(tf.global_variables_initializer())

def printStats(what):
    train_cost, train_accuracy, train_argmaxed_Y = sess.run([loss, accuracy, argmaxed_Y], 
                                       feed_dict={X: check_trainFeatures,
                                                  labels: check_trainLabels_onehot,
                                                  keep_prob_in: 1.0,
                                                  keep_prob_hid: 1.0})
    test_cost, test_accuracy, text_argmaxed_Y = sess.run([loss, accuracy, argmaxed_Y], 
                                     feed_dict={X: testFeatures,
                                                labels: testLabels_onehot,
                                                keep_prob_in: 1.0,
                                                keep_prob_hid: 1.0})
    _, train_counts = np.unique(train_argmaxed_Y, return_counts=True)
    _, test_counts = np.unique(text_argmaxed_Y, return_counts=True)
    print("{}\n  TRAIN cost/acc:{:.4f}/{:.4f}% {}, TEST cost/acc:{:.4f}/{:.4f}% {}"\
          .format(what, train_cost, train_accuracy*100, train_counts,
                  test_cost, test_accuracy*100, test_counts))

printStats("BEFORE Training")
print("-----------------------------------\n")


BEFORE Training
  TRAIN cost/acc:3.0087/46.8049% [  26 4700], TEST cost/acc:3.0626/39.7799% [  15 4711]
-----------------------------------



In [49]:
test_cost, test_accuracy, test_Y, argmaxed_test_Y, softmaxed_test_Y = sess.run([loss, accuracy, Y, argmaxed_Y, softmaxed_Y], 
                                     feed_dict={X: testFeatures,
                                                labels: testLabels_onehot,
                                                keep_prob_in: 1.0,
                                                keep_prob_hid: 1.0})
test_Y[1010:1030]

array([[-2.09142232,  2.47218537],
       [-2.2506628 ,  2.4723506 ],
       [-1.54527867,  2.98627281],
       [-1.46618009,  3.928267  ],
       [-1.70576215,  3.9822793 ],
       [-1.6509701 ,  3.26231194],
       [-1.12588501,  3.42122102],
       [-1.3550725 ,  3.69757462],
       [-1.23799002,  4.5408802 ],
       [-0.3111079 ,  4.38337564],
       [-1.09018695,  4.20815468],
       [-0.78365612,  4.59549856],
       [-2.31993842,  5.63417053],
       [-0.71542501,  4.9671073 ],
       [-1.10279167,  5.01119947],
       [-1.51306736,  5.21146631],
       [-1.5462954 ,  5.79684401],
       [-2.65777969,  6.09949923],
       [-2.08571243,  4.94816351],
       [-1.16707909,  3.0348568 ]], dtype=float32)

In [50]:
argmaxed_test_Y[1010:1030]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [51]:
time_start = timer()
epoch = 0
for mini_batch in range(1, total_mini_batches+1):
    batch_inputs, batch_labels = getTrainMiniBatch(mini_batch_size)
    sess.run([train, accuracy], feed_dict={X: batch_inputs, 
                                           labels: batch_labels,
                                           keep_prob_in: keep_prob_in_train,
                                           keep_prob_hid: keep_prob_hid_train})
    if ((epoch == 0) and (mini_batch % 100 == 0) and (mini_batch <= 1000)):
        printStats(' ... progress: In "Epoch" {} of {}, mini batch {} of {}'.format(epoch, epochs, mini_batch, total_mini_batches))
    if (((epoch > 0) or (mini_batch > 1000)) and (mini_batch % 500 == 0)):
        print(' ... progress: In "Epoch" {} of {}, mini batch {} of {}'.format(epoch, epochs, mini_batch, total_mini_batches))
        
    if (mini_batch % mini_batches_per_epoch == 0):
        epoch += 1
        printStats('\nAfter "Epoch" {} of {}, mini batch {} of {}'.format(epoch, epochs, mini_batch, total_mini_batches))
        
training_time = timer()-time_start
printStats("\nFINISHED")
print("TRAINING took {} seconds.".format(training_time))

Shuffling training features and labels - 0..
 ... progress: In "Epoch" 0 of 100, mini batch 100 of 51800
  TRAIN cost/acc:0.7231/53.0258% [4682   44], TEST cost/acc:0.6876/60.0719% [4670   56]
 ... progress: In "Epoch" 0 of 100, mini batch 200 of 51800
  TRAIN cost/acc:0.6892/53.6394% [3467 1259], TEST cost/acc:0.6909/56.1997% [3673 1053]
 ... progress: In "Epoch" 0 of 100, mini batch 300 of 51800
  TRAIN cost/acc:0.7067/51.6293% [2394 2332], TEST cost/acc:0.6961/53.3220% [2095 2631]
 ... progress: In "Epoch" 0 of 100, mini batch 400 of 51800
  TRAIN cost/acc:0.7087/51.8832% [2786 1940], TEST cost/acc:0.7036/52.8354% [3140 1586]
 ... progress: In "Epoch" 0 of 100, mini batch 500 of 51800
  TRAIN cost/acc:0.6954/54.9090% [4059  667], TEST cost/acc:0.6836/58.2945% [4152  574]

After "Epoch" 1 of 100, mini batch 518 of 51800
  TRAIN cost/acc:0.6967/52.4122% [2141 2585], TEST cost/acc:0.7007/51.7562% [2101 2625]
Shuffling training features and labels - 1..
 ... progress: In "Epoch" 1 of 10


After "Epoch" 31 of 100, mini batch 16058 of 51800
  TRAIN cost/acc:0.3440/81.2949% [3154 1572], TEST cost/acc:1.6021/53.8934% [3068 1658]
Shuffling training features and labels - 31..
 ... progress: In "Epoch" 31 of 100, mini batch 16500 of 51800

After "Epoch" 32 of 100, mini batch 16576 of 51800
  TRAIN cost/acc:0.3312/82.5010% [3045 1681], TEST cost/acc:1.8111/53.9568% [3039 1687]
Shuffling training features and labels - 32..
 ... progress: In "Epoch" 32 of 100, mini batch 17000 of 51800

After "Epoch" 33 of 100, mini batch 17094 of 51800
  TRAIN cost/acc:0.2949/84.6381% [2934 1792], TEST cost/acc:1.8307/52.8354% [2984 1742]
Shuffling training features and labels - 33..
 ... progress: In "Epoch" 33 of 100, mini batch 17500 of 51800

After "Epoch" 34 of 100, mini batch 17612 of 51800
  TRAIN cost/acc:0.2982/84.5112% [2928 1798], TEST cost/acc:2.0100/53.1951% [2869 1857]
Shuffling training features and labels - 34..
 ... progress: In "Epoch" 34 of 100, mini batch 18000 of 51800

Aft

 ... progress: In "Epoch" 63 of 100, mini batch 33000 of 51800

After "Epoch" 64 of 100, mini batch 33152 of 51800
  TRAIN cost/acc:0.1658/91.8536% [2755 1971], TEST cost/acc:3.0207/53.2374% [2847 1879]
Shuffling training features and labels - 64..
 ... progress: In "Epoch" 64 of 100, mini batch 33500 of 51800

After "Epoch" 65 of 100, mini batch 33670 of 51800
  TRAIN cost/acc:0.1704/91.5996% [2809 1917], TEST cost/acc:3.2911/52.7084% [2994 1732]
Shuffling training features and labels - 65..
 ... progress: In "Epoch" 65 of 100, mini batch 34000 of 51800

After "Epoch" 66 of 100, mini batch 34188 of 51800
  TRAIN cost/acc:0.1455/93.0597% [2708 2018], TEST cost/acc:3.1633/52.6873% [2843 1883]
Shuffling training features and labels - 66..
 ... progress: In "Epoch" 66 of 100, mini batch 34500 of 51800

After "Epoch" 67 of 100, mini batch 34706 of 51800
  TRAIN cost/acc:0.1354/93.5252% [2658 2068], TEST cost/acc:3.2543/52.3487% [2701 2025]
Shuffling training features and labels - 67..
 ...

 ... progress: In "Epoch" 96 of 100, mini batch 50000 of 51800

After "Epoch" 97 of 100, mini batch 50246 of 51800
  TRAIN cost/acc:0.1158/94.8582% [2579 2147], TEST cost/acc:4.0228/51.9467% [2666 2060]
Shuffling training features and labels - 97..
 ... progress: In "Epoch" 97 of 100, mini batch 50500 of 51800

After "Epoch" 98 of 100, mini batch 50764 of 51800
  TRAIN cost/acc:0.1029/95.3237% [2615 2111], TEST cost/acc:4.2083/51.6928% [2758 1968]
Shuffling training features and labels - 98..
 ... progress: In "Epoch" 98 of 100, mini batch 51000 of 51800

After "Epoch" 99 of 100, mini batch 51282 of 51800
  TRAIN cost/acc:0.0839/96.4029% [2612 2114], TEST cost/acc:4.1910/51.6293% [2677 2049]
Shuffling training features and labels - 99..
 ... progress: In "Epoch" 99 of 100, mini batch 51500 of 51800

After "Epoch" 100 of 100, mini batch 51800 of 51800
  TRAIN cost/acc:0.0845/96.5510% [2619 2107], TEST cost/acc:4.4877/53.5336% [2953 1773]

FINISHED
  TRAIN cost/acc:0.0845/96.5510% [2619 